In [17]:
# Importa as bibliotecas necessárias
import gensim
import gensim.corpora as corpora
import pandas as pd
import re
import spacy

from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaModel
from gensim.models import LdaMulticore

from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

from tqdm import tqdm
tqdm.pandas()

In [14]:
# Carrega o dataset referente às reviews do Airbnb Rio de Janeiro
reviews = pd.read_csv("data/reviews.csv")

In [15]:
# Apresenta as primeiras 5 reviews
reviews.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r\n...
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...


In [4]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292516 entries, 0 to 292515
Data columns (total 7 columns):
listing_id       292516 non-null int64
id               292516 non-null int64
date             292516 non-null object
reviewer_id      292516 non-null int64
reviewer_name    292516 non-null object
comments         292446 non-null object
language         292076 non-null object
dtypes: int64(3), object(4)
memory usage: 15.6+ MB


In [5]:
# Converte coluna comments para string
reviews["comments"] = reviews["comments"].astype(str)

In [ ]:
# Adiciona coluna com idioma da review
def try_detect_language(text):
    """
    Dado uma string text, tenta obter o idioma predominante.
    
    Argumentos:
    text -- (str) o texto para o qual se deseja detectar o idioma
    Retorna: (str) o código do idioma (e.g. 'en' para inglês). 
             Retorna uma string vazia caso não seja possível detectar o idioma
    """
    try:
        return detect(text)
    except:
        return ""

reviews["language"] = reviews["comments"].apply(try_detect_language)

In [ ]:
# Verifica total de reviews para cada idioma identificado
reviews.language.value_counts().head()

## Preprocessamento dos Dados

In [6]:
nlp_pt = spacy.load("pt")

def preprocess(text, 
               min_token_len = 2, 
               irrelevant_pos = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE'],
               nlp = nlp_pt): 
    """
    Dado text, min_token_len, e irrelevant_pos, faz o preprocessamento do texto
    
    Argumentos:
    text -- (str) texto a ser pré-processado
    min_token_len -- (int) o menor comprimento de token a ser considerado
    irrelevant_pos -- (list) lista as classes gramaticais a serem ignoradas
    
    Retorna: (str) texto pré processado
    """
    
    # Converte todo os texto para apenas caractéres minúsculos
    text = text.lower()
    
    # Remove termos utilizados pelo airbnb para ocultar informações
    text = re.sub(r'\((.*hidden by airbnb)\)', "", text)
    
    # Processa o texto usando spacy
    doc = nlp(text)
    
    preprocessed = []
    
    # Remove stop words, pontuação, tokens menores que min_token_len e tokens cuja 
    # classe gramatical identificada é irrelavante para análise
    for token in doc:
        if not token.is_stop and not token.is_punct and len(token) >= min_token_len and token.pos_ not in irrelevant_pos:
            # Armazena o lemma do token na variável de saída
            preprocessed.append(token.lemma_)
            
    return " ".join(preprocessed)

def preprocess_pt(text):
    return preprocess(text, nlp = nlp_pt)

In [7]:
reviews_pt = reviews[reviews.language == "pt"]

In [8]:
reviews_pt["clean_comments"] = reviews_pt.comments.progress_apply(preprocess_pt)

100%|██████████| 138315/138315 [1:07:43<00:00, 34.04it/s]    
/Users/marcelle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
reviews_pt.to_csv('data/reviews_pt_preprocessed.csv', index = False)

In [46]:
preprocessed_corpus = [doc.split() for doc in reviews_pt.clean_comments.tolist()]
# Cria um vocabulário para o modelo LDA e converte
# nosso corpus em uma matriz documento-termo para LDA
dictionary = corpora.Dictionary(preprocessed_corpus)

# Opcional: Remove palavras que aparecem em menos de no_below reviews e em mais do que no_above % das reviews
dictionary.filter_extremes(no_below = 10, no_above = 0.1, keep_n = 10000)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in preprocessed_corpus]

In [47]:
print(dictionary)

Dictionary(5888 unique tokens: ['apt', 'conhecer', 'hotel', 'maneiro', 'maravilhoso']...)


In [48]:
num_topics = range(5, 14)
passes = 10

lda_models = {}

for n in num_topics:
    print('Number of topics = ', n)
    
    lda = LdaMulticore(corpus = doc_term_matrix,
                       id2word = dictionary,
                       num_topics = n,
                       workers = 3,
                       passes = passes)
    
    lda_models[str(n)] = lda

    print(lda.print_topics(), '\n')
    print("Coherence:", cm.get_coherence())
    
    cm = CoherenceModel(model = lda, 
                        texts = preprocessed_corpus,
                        dictionary = dictionary,
                        coherence = 'c_v')

Number of topics =  5
[(0, '0.022*"aptar" + 0.022*"foto" + 0.017*"organizar" + 0.016*"prestativo" + 0.015*"responder" + 0.015*"aconchegante" + 0.013*"rápido" + 0.012*"condomínio" + 0.012*"adorar" + 0.010*"indicar"'), (1, '0.025*"cama" + 0.018*"cozinhar" + 0.014*"banheiro" + 0.013*"problema" + 0.013*"ar" + 0.012*"condicionar" + 0.011*"roupar" + 0.010*"haver" + 0.010*"ter" + 0.009*"dia"'), (2, '0.062*"check" + 0.048*"in" + 0.041*"horário" + 0.035*"flexível" + 0.030*"check-in" + 0.027*"out" + 0.020*"preço" + 0.019*"host" + 0.018*"check-out" + 0.017*"combinar"'), (3, '0.023*"maravilhoso" + 0.017*"incrível" + 0.016*"sentir" + 0.015*"visto" + 0.013*"pessoa" + 0.013*"deixar" + 0.012*"receber" + 0.012*"experiência" + 0.011*"aconchegante" + 0.011*"vontade"'), (4, '0.030*"metrô" + 0.021*"bar" + 0.020*"supermercado" + 0.019*"mercar" + 0.019*"fácil" + 0.017*"copacabana" + 0.017*"acesso" + 0.016*"farmácia" + 0.013*"tranquilo" + 0.012*"padaria"')] 

Coherence: 0.5618693571383846
Number of topics =  

Coherence: 0.5873607644226186
Number of topics =  11
[(0, '0.020*"dia" + 0.016*"cidade" + 0.013*"noite" + 0.013*"chegar" + 0.012*"bairro" + 0.012*"tranquilo" + 0.011*"acesso" + 0.011*"passar" + 0.011*"lapa" + 0.010*"centrar"'), (1, '0.039*"deixar" + 0.038*"dica" + 0.035*"dar" + 0.030*"ajudar" + 0.029*"vontade" + 0.022*"dúvida" + 0.020*"receber" + 0.018*"precisar" + 0.015*"chegar" + 0.014*"café"'), (2, '0.060*"foto" + 0.028*"prestativo" + 0.027*"fácil" + 0.026*"organizar" + 0.022*"simpático" + 0.020*"agradável" + 0.020*"tranquilo" + 0.020*"aptar" + 0.020*"acesso" + 0.016*"educar"'), (3, '0.062*"metrô" + 0.041*"supermercado" + 0.040*"mercar" + 0.039*"bar" + 0.033*"farmácia" + 0.025*"padaria" + 0.021*"quadrar" + 0.021*"estação" + 0.020*"copacabana" + 0.018*"comércio"'), (4, '0.050*"visto" + 0.042*"condomínio" + 0.024*"piscina" + 0.021*"estruturar" + 0.019*"barrir" + 0.018*"flat" + 0.018*"preço" + 0.016*"lindo" + 0.015*"frente" + 0.014*"prédio"'), (5, '0.047*"check" + 0.044*"rápido" + 0.03

Analisando o Coherence score, o melhor modelo seria aquele treinado com `num_topics = 12`. Entretanto, analisando a interpretabilidade dos tópicos identificados, esse modelo deixa a desejar, uma vez que há uma sobreposição de temas em diferentes tópicos. Assim, é preferível selecionar um modelo com menor número de tópicos, que vai oferecer resultados mais interpretáveis

In [50]:
lda_model = lda_models["5"]